# NoSQL (HBase) (sesión 6)

![Image of HBase](http://hbase.apache.org/images/hbase_logo_with_orca_large.png)

Esta hoja muestra cómo acceder a bases de datos HBase y también a conectar la salida con Jupyter.

Se puede utilizar el *shell* propio de HBase en la máquina virtual llamando al programa:

    $ hbase/bin/hbase shell

La diferencia es que ese programa espera código Ruby y aquí trabajaremos con Python.

### Nota sobre la caída de RegionServers

En este entorno con poca memoria son frecuentes las caídas de RegionServers. Sería conveniente:

- dar a la memoria virtual al menos 3GB de memoria,
- aumentar el tamaño del HEAP de los procesos de HBase, y
- aumentar el tiempo de _timeout_ de Zookeeper.

En la [documentación de HBase](http://hbase.apache.org/book.html#trouble.rs) dan unas recomendaciones, sobre todo, para carga inicial, como he realizado estos días para cargar las bases de datos de ejemplo:

> Make sure you give plenty of RAM (in hbase-env.sh), the default of 1GB won’t be able to sustain long running imports.
>
> [...]
>
> If this is happening during an upload which only happens once (like initially loading all your data into HBase), consider bulk loading.

Aunque no usamos _bulk loading_ para mostrar cómo se añaden datos desde Python (el _bulk loading_ hay que hacerlo en Java).

Las caídas en los RegionServers pueden producirse por varias cuestiones: falta de memoria, timeout por la ejecución del GC de Java, etc. Estas caídas son aceptadas como normales por el sistema HBase, que continuará funcionando con el resto de RegionServers y aceptará un RegionServer que terminó abruptamente una vez reiniciado.

En nuestra VM sólo hay un RegionServer, y se puede iniciar si cayó con el comando:

    ~/hbase/bin/start-daemon.sh start regionserver

El siguiente _script_ comprueba cada 30 segundos la salida de depuración del Máster de HBase, y si ve que no hay RegionServers, llama al script de reinicio del único RegionServer. El cliente continuará sin problemas después de unos segundos de inicialización. Al cabo del tiempo, los RegionServer que no funcionan se eliminan por HBase.

In [1]:
%%writefile restart-regionserver.sh
#! /bin/sh
while true ; do
	sleep 30 # Sleep before to give time HBase to start
	ns=`curl -s http://localhost:60010/jmx | grep numRegionServers | tr -cd [0-9]`
	test -z "$ns" || test $ns -gt 0 || ~/hbase/bin/hbase-daemon.sh start regionserver 
done


Overwriting restart-regionserver.sh


### Memoria de intercambio

El tamaño de la memoria que requiere puntualmente HBase hace que tengamos que crear un fichero de intercambio si no existe, y activarla. Se usarán 4GB para permitir el uso de memoria. Esto hará el sistema lento en caso de que tenga que hacer uso del intercambio, pero al menos no morirán por falta de memoria los distintos servidores de HBase.

In [2]:
%%bash
if ! sudo grep /swap /proc/swaps 2>&1 >/dev/null
then
    sudo fallocate -l 4GiB /swap
    sudo chmod 0600 /swap
    sudo mkswap /swap
    sudo swapon /swap
fi

Setting up swapspace version 1, size = 4 GiB (4294963200 bytes)
no label, UUID=c3aa0d80-ae79-4d08-9704-fdd7f87e74e9


Iniciamos HBase. Esto lanza todos los demonios y el demonio de HDFS.

In [3]:
%%bash
~/start-hbase.sh

Starting namenodes on [localhost]
localhost: Warning: Permanently added 'localhost' (ECDSA) to the list of known hosts.
localhost: starting namenode, logging to /home/vagrant/hadoop-2.6.4/logs/hadoop-vagrant-namenode-vagrant-bdge.out
localhost: starting datanode, logging to /home/vagrant/hadoop-2.6.4/logs/hadoop-vagrant-datanode-vagrant-bdge.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: Warning: Permanently added '0.0.0.0' (ECDSA) to the list of known hosts.
0.0.0.0: starting secondarynamenode, logging to /home/vagrant/hadoop-2.6.4/logs/hadoop-vagrant-secondarynamenode-vagrant-bdge.out
localhost: starting zookeeper, logging to /home/vagrant/hbase-1.1.6/bin/../logs/hbase-vagrant-zookeeper-vagrant-bdge.out
starting master, logging to /home/vagrant/hbase-1.1.6/bin/../logs/hbase-vagrant-master-vagrant-bdge.out
OpenJDK 64-Bit Server VM warning: ignoring option PermSize=128m; support was removed in 8.0
OpenJDK 64-Bit Server VM warning: ignoring option MaxPermSize=128m; support was remo

Iniciamos el script en segundo plano para que reinicie los regionservers que se caen:

In [4]:
%%bash --bg
sh restart-regionserver.sh

Starting job # 0 in a separate thread.


In [5]:
# Config
%env DIR=/vagrant

env: DIR=/vagrant


In [6]:
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

Usaremos la librería `happybase` para python. La cargamos a continuación y hacemos la conexión.

In [7]:
import happybase

host = '127.0.0.1'
connection = happybase.Connection(host)
connection.tables()

[]

In [8]:
%%bash
FILE=Posts.csv
(test -e $DIR/$FILE && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/$FILE.gz -O - 2>/dev/null | gunzip > $DIR/$FILE) \
  && echo OK)

Ya descargado


In [9]:
%%bash
FILE=Users.csv
(test -e $DIR/$FILE && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/$FILE.gz -O - 2>/dev/null | gunzip > $DIR/$FILE) \
  && echo OK)

Ya descargado


In [10]:
%%bash
FILE=Tags.csv
(test -e $DIR/$FILE && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/$FILE.gz -O - 2>/dev/null | gunzip > $DIR/$FILE) \
  && echo OK)

Ya descargado


In [11]:
%%bash
FILE=Comments.csv
(test -e $DIR/$FILE && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/$FILE.gz -O - 2>/dev/null | gunzip > $DIR/$FILE) \
  && echo OK)

Ya descargado


In [12]:
%%bash
FILE=Votes.csv
(test -e $DIR/$FILE && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/$FILE.gz -O - 2>/dev/null | gunzip > $DIR/$FILE) \
  && echo OK)

Ya descargado


In [13]:
%%bash
FILE=PostHistory.csv
(test -e $DIR/$FILE && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/$FILE.gz -O - 2>/dev/null | gunzip > $DIR/$FILE) \
  && echo OK)

OK


Para la carga inicial, vamos a crear todas las tablas con una única familia de columnas, `rawdata`, donde meteremos toda la información _raw_ comprimida. Después podremos hacer reorganizaciones de los datos para hacer el acceso más eficiente. Es una de las muchas ventajas de no tener un esquema.

In [14]:
# Create tables
tables = ['posts', 'votes', 'users', 'tags', 'comments']
for t in tables:
    try:
        connection.create_table(
            t,
            {
                'rawdata': dict(max_versions=1,compression='GZ')
            })
    except:
        print "Database already exists: {0}.".format(t)
        pass
connection.tables()

['comments', 'posts', 'tags', 'users', 'votes']

El código de importación es siempre el mismo, ya que se coge la primera fila del CSV que contiene el nombre de las columnas y se utiliza para generar nombres de columnas dentro de la familia de columnas dada como parámetro. La función `csv_to_hbase()` acepta un fichero CSV a abrir, un nombre de tabla y una familia de columnas donde agregar las columnas del fichero CSV. En nuestro caso siempre va a ser `rawdata`.

In [15]:
import csv

def csv_to_hbase(file, tablename, cf):
    table = connection.table(tablename)
    
    with open(file) as f:
        # La llamada csv.reader() crea un iterador sobre un fichero CSV
        reader = csv.reader(f, dialect='excel')
        
        # Se leen las columnas. Sus nombres se usarán para crear las diferentes columnas en la familia
        columns = reader.next()
        columns = [cf + ':' + c for c in columns]
        
        with table.batch(batch_size=50) as b:
            for row in reader:
                # La primera columna se usará como Row Key
                b.put(row[0], dict(zip(columns[1:], row[1:])))


In [16]:
import os

for t in tables:
    print "Importando tabla {0}...".format(t)
    %time csv_to_hbase(os.environ['DIR']+'/'+ t.capitalize() + '.csv', t, 'rawdata')

Importando tabla posts...
CPU times: user 2.61 s, sys: 836 ms, total: 3.45 s
Wall time: 24.8 s
Importando tabla votes...
CPU times: user 1.77 s, sys: 152 ms, total: 1.92 s
Wall time: 10.6 s
Importando tabla users...
CPU times: user 968 ms, sys: 168 ms, total: 1.14 s
Wall time: 6.53 s
Importando tabla tags...
CPU times: user 24 ms, sys: 8 ms, total: 32 ms
Wall time: 187 ms
Importando tabla comments...
CPU times: user 1.13 s, sys: 256 ms, total: 1.38 s
Wall time: 6.91 s


### Construcción de estructuras anidadas

Al igual que pasaba con MongoDB, las bases de datos NoSQL como en este caso HBase permiten almacenar estructuras de datos complejas. En nuestro caso vamos a agregar los comentarios de cada pregunta o respuesta (post) en columnas del mismo. Para ello, creamos una nueva familia de columnas `comments`.

HBase es bueno para añadir columnas sencillas, por ejemplo que contengan un valor. Sin embargo, si queremos añadir objetos complejos, tenemos que jugar con la codificación de la familia de columnas y columna.

Usaremos el shell porque `happybase` no permite alterar tablas ya creadas.

In [17]:
%%bash
cat <<EOF | ~/hbase/bin/hbase shell

disable 'posts'

alter 'posts', {NAME => 'comments', VERSIONS => 1}

enable 'posts'

EOF

HBase Shell; enter 'help<RETURN>' for list of supported commands.
Type "exit<RETURN>" to leave the HBase Shell
Version 1.1.6, r2039d967835afd066676b83f0bcb722cc55fdde3, Sun Aug 28 10:36:13 PDT 2016


disable 'posts'
0 row(s) in 4.7620 seconds


alter 'posts', {NAME => 'comments', VERSIONS => 1}
Updating all regions with the new schema...
1/1 regions updated.
Done.
0 row(s) in 1.9730 seconds


enable 'posts'
0 row(s) in 1.3990 seconds




SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/vagrant/hbase-1.1.6/lib/slf4j-log4j12-1.7.5.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/vagrant/hadoop-2.6.4/share/hadoop/common/lib/slf4j-log4j12-1.7.5.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


Cada comentario que añadimos contiene, al menos:

- un id único
- un texto
- un autor
- etc.

¿Cómo se consigue meterlo en una única familia de columnas?

Hay varias formas. La que usaremos aquí, añadiremos el **id** de cada comentario como parte del nombre de la columna. Por ejemplo, el comentario con Id 2000, generará las columnas:

- `Id_2000` (valor 2000)
- `UserId_2000`
- `PostId_2000`
- `Text_2000`

con sus correspondientes valores. Así, todos los datos relativos al comentario con Id original 2000, estarán almacenados en todas las columnas que termienn en `_2000`. La base de datos permite implementar filtros que nos permiten buscar esto de forma muy sencilla. Los veremos después.

In [18]:
comments = connection.table('comments')
posts = connection.table('posts')

with posts.batch(batch_size=50) as b:
    # Hacer un scan de la tabla
    for key, data in comments.scan():
        comment = {'comments:' + d.split(':')[1] + "_" + str(key): data[d] for d in data.keys()}
        b.put(data['rawdata:PostId'], comment)

El siguiente código permite mostrar de forma amigable las tablas extraídas de la base de datos en forma de diccionario:

In [19]:
# http://stackoverflow.com/a/30525061/62365
class DictTable(dict):
    # Overridden dict class which takes a dict in the form {'a': 2, 'b': 3},
    # and renders an HTML Table in IPython Notebook.
    def _repr_html_(self):
        html = ["<table width=100%>"]
        for key, value in self.iteritems():
            html.append("<tr>")
            html.append("<td>{0}</td>".format(key))
            html.append("<td>{0}</td>".format(value))
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

In [20]:
# Muestra cómo queda la fila del Id del Post 9997
posts = connection.table('posts')
DictTable(posts.row('9997'))

rawdata:CommunityOwnedDate,
rawdata:Tags,
comments:Text_17504,"Efectivamente era eso, vaya tontería de problema, no se como no se me ocurrio; muchísimas gracias."
rawdata:LastEditorDisplayName,
rawdata:OwnerUserId,6485
rawdata:Body,"He estado revisando tu código y no encuentro un error tan claro, lo que observo es que cuando escribes un nombre por ejemplo ac/dclo haces en minusculas, y en la segunda pantalla esta: AC/DChas probado poner el nombre tal y como esta en la segunda pantalla? Tal vez por eso no se selecciona, con que una sola letra sea diferente ya no funcionaria ya que estas seleccionando un item mediante elegirArtista.setSelectedItem(campoTextoArtista2.getText());"
rawdata:CreationDate,2016-05-16T02:16:06.127
comments:Score_17504,0
rawdata:LastEditorUserId,
rawdata:Title,
comments:PostId_17504,9997


## Wikipedia

Como otro ejemplo de carga de datos y de organización en HBase, veremos de manera simplificada el ejemplo de la wikipedia visto en teoría.

A continuación se descarga una pequeña parte del fichero de la wikipedia en XML:

In [21]:
%%bash
FILE=eswiki.xml
(test -e $DIR/$FILE && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/wikipedia/$FILE.gz -O - 2>/dev/null | gunzip > $DIR/$FILE) \
  && echo OK)

OK


In [22]:
!head -200 $DIR/eswiki.xml

<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="es">
  <siteinfo>
    <sitename>Wikipedia</sitename>
    <dbname>eswiki</dbname>
    <base>https://es.wikipedia.org/wiki/Wikipedia:Portada</base>
    <generator>MediaWiki 1.28.0-wmf.22</generator>
    <case>first-letter</case>
    <namespaces>
      <namespace key="-2" case="first-letter">Medio</namespace>
      <namespace key="-1" case="first-letter">Especial</namespace>
      <namespace key="0" case="first-letter" />
      <namespace key="1" case="first-letter">Discusión</namespace>
      <namespace key="2" case="first-letter">Usuario</namespace>
      <namespace key="3" case="first-letter">Usuario discusión</namespace>
      <namespace key="4" case="first-letter">Wikipedia</namespace>
      <namespace key="5" case="first-letter">Wikipedia 

Se crea la tabla para albergar la `wikipedia`. Igual que la vista en teoría, pero aquí se usa `wikipedia` en vez de `wiki` para que no colisionen la versión completa con la reducida.

In [23]:
%%bash
cat <<EOF | ~/hbase/bin/hbase shell
create 'wikipedia' , 'text', 'revision'

disable 'wikipedia' # Para evitar su uso temporal

alter 'wikipedia' , { NAME => 'text', VERSIONS => org.apache.hadoop.hbase.HConstants::ALL_VERSIONS }

alter 'wikipedia' , { NAME => 'revision', VERSIONS => org.apache.hadoop.hbase.HConstants::ALL_VERSIONS }

alter 'wikipedia' , { NAME => 'text', COMPRESSION => 'GZ', BLOOMFILTER => 'ROW'}

enable 'wikipedia'

EOF

HBase Shell; enter 'help<RETURN>' for list of supported commands.
Type "exit<RETURN>" to leave the HBase Shell
Version 1.1.6, r2039d967835afd066676b83f0bcb722cc55fdde3, Sun Aug 28 10:36:13 PDT 2016

create 'wikipedia' , 'text', 'revision'
0 row(s) in 1.6390 seconds

Hbase::Table - wikipedia

disable 'wikipedia' # Para evitar su uso temporal
0 row(s) in 2.3890 seconds


alter 'wikipedia' , { NAME => 'text', VERSIONS => org.apache.hadoop.hbase.HConstants::ALL_VERSIONS }
Updating all regions with the new schema...
1/1 regions updated.
Done.
0 row(s) in 2.0430 seconds


alter 'wikipedia' , { NAME => 'revision', VERSIONS => org.apache.hadoop.hbase.HConstants::ALL_VERSIONS }
Updating all regions with the new schema...
1/1 regions updated.
Done.
0 row(s) in 1.9370 seconds


alter 'wikipedia' , { NAME => 'text', COMPRESSION => 'GZ', BLOOMFILTER => 'ROW'}
Updating all regions with the new schema...
1/1 regions updated.
Done.
0 row(s) in 1.9440 seconds


enable 'wikipedia'
0 row(s) in 1.3670 sec

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/vagrant/hbase-1.1.6/lib/slf4j-log4j12-1.7.5.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/vagrant/hadoop-2.6.4/share/hadoop/common/lib/slf4j-log4j12-1.7.5.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


Este código, visto en teoría, recorre el árbol XML construyendo documentos y llamando a la función `callback` con cada uno. Los documentos son diccionarios con las claves encontradas dentro de los tags `<page>...</page>`.

In [24]:
import xml.sax
import re

class WikiHandler(xml.sax.handler.ContentHandler):

    def __init__(self):
        self._charBuffer = ''
        self.document = {}

    def _getCharacterData(self):
        data = self._charBuffer
        self._charBuffer = ''
        return data

    def parse(self, f, callback):
        self.callback = callback
        xml.sax.parse(f, self)

    def characters(self, data):
        self._charBuffer = self._charBuffer + data

    def startElement(self, name, attrs):
        if name == 'page':
        # print 'Start of page'
            self.document = {}
        if re.match(r'title|timestamp|username|comment|text', name):
            self._charBuffer = ''

    def endElement(self, name):
        if re.match(r'title|timestamp|username|comment|text', name):
            self.document[name] = self._getCharacterData()
            # print(name, ': ', self.document[name][:20])
        if 'revision' == name:
            self.callback(self.document)


El codigo a continuación, cada vez que el código anterior llama a la función `processdoc()` se añade un documento a la base de datos.

In [25]:
import time
import os

class FillWikiTable():
    """Llena la tabla Wiki"""
    def __init__(self):
        # Conectar a la base de datos a través de Thrift
        self.table = connection.table('wikipedia')

    def run(_s):
        def processdoc(d):
            print "Callback called with", d['title']
            tuple_time = time.strptime(d['timestamp'], "%Y-%m-%dT%H:%M:%SZ")
            timestamp = int(time.mktime(tuple_time))
            _s.table.put(d['title'],
                         {'text:': d.get('text',''),
                          'revision:author': d.get('username',''),
                          'revision:comment': d.get('comment','')},
                         timestamp=timestamp)

        with open(os.environ['DIR']+'/'+'eswiki.xml','rb') as f:
            start = time.time()
            WikiHandler().parse(f, processdoc)
            end = time.time()
            print ("End adding documents. Time: %.5f" % (end - start))

In [26]:
FillWikiTable().run()

Callback called with Wikipedia:Artículos solicitados
Callback called with Andorra
Callback called with Argentina
Callback called with Geografía de Andorra
Callback called with Demografía de Andorra
Callback called with Comunicaciones de Andorra
Callback called with Artes visuales
Callback called with Agricultura
Callback called with Astronomía galáctica
Callback called with ASCII
Callback called with Arquitectura
Callback called with Anoeta
Callback called with Ana María Matute
Callback called with Agujero negro
Callback called with Anarquía
Callback called with América del Norte
Callback called with América del Sur
Callback called with Asia
Callback called with Año
Callback called with Asaph Hall
Callback called with Afganistán
Callback called with Arqueología
Callback called with Wikipedia:Anuncios
Callback called with Commodore Amiga
Callback called with Commodore Amiga 500
Callback called with Francesc Aguilar Villalonga
Callback called with Aquifoliaceae
Callback called with Sapin

El código a continuación permite ver las diferentes versiones de una revisión. Como la versión reducida es muy pequeña no da lugar a que haya ninguna revisión, pero con este código se vería. Hace uso del _shell_ de HBase.

In [27]:
%%bash
cat <<EOF | ~/hbase/bin/hbase shell

get 'wikipedia', 'Commodore Amiga', {COLUMN => 'revision',VERSIONS=>10}

EOF

HBase Shell; enter 'help<RETURN>' for list of supported commands.
Type "exit<RETURN>" to leave the HBase Shell
Version 1.1.6, r2039d967835afd066676b83f0bcb722cc55fdde3, Sun Aug 28 10:36:13 PDT 2016


get 'wikipedia', 'Commodore Amiga', {COLUMN => 'revision',VERSIONS=>10}
COLUMN  CELL
 revision:author timestamp=1474981078, value=
 revision:comment timestamp=1474981078, value=
2 row(s) in 0.3810 seconds




SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/vagrant/hbase-1.1.6/lib/slf4j-log4j12-1.7.5.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/vagrant/hadoop-2.6.4/share/hadoop/common/lib/slf4j-log4j12-1.7.5.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


### Enlazado de documentos en la wikipedia

Los artículos de la wikipedia llevan enlaces entre sí, incluyendo referencias del tipo `[[artículo referenciado]]`. Se pueden extraer estos enlaces y se puede construir un grafo de conexiones. Para cada artículo, se anotarán qué enlaces hay que salen de él y hacia qué otros artículos enlazan y también qué enlaces llegan a él. Esto se hará con dos familias de columnas, `from` y `to`. 

En cada momento, se añadirá una columna `from:artículo` cuando un artículo nos apunte, y otras columnas `to:articulo` con los artículos que nosotros enlazamos.

In [28]:
import sys

class BuildLinks():
    """Llena la tabla de Links"""
    def __init__(self):
        # Create table
        try:
            connection.create_table(
                "wikilinks",
                {
                    'from': dict(bloom_filter_type='ROW',max_versions=1),
                    'to' : dict(bloom_filter_type='ROW',max_versions=1)
                })
        except:
            print "Database wikilinks already exists."
            pass

        self.table = connection.table('wikilinks')
        self.wikitable = connection.table('wikipedia')

    def run(self):
        print "run";
        linkpattern = r'\[\[([^\[\]\|\:\#][^\[\]\|:]*)(?:\|([^\[\]\|]+))?\]\]'
        # target, label

        with self.table.batch(batch_size=50) as b:
            for key, data in self.wikitable.scan():
                to_dict = {}
                doc = key.strip()
                print "\n", doc, ":"
                for mo in re.finditer(linkpattern, data['text:']):
                    (target, label) = mo.groups()

                    target = target.strip()

                    if target == '':
                        continue

                    label = '' if not label else label
                    label = label.strip()

                    to_dict['to:' + target] = label

                    sys.stdout.write(".")
                    #print "%s -> %s (%s)" % (doc, target, label)
                    
                    b.put(target, {'from:' + doc : label})

                if bool(to_dict):
                    b.put(doc, to_dict)


In [29]:
BuildLinks().run()

run

A fala :
.
AGA :
.....
ALGOL :
.................................
ASCII :
.......................................................................................................................................................................................................
Abreviatura :
................................
Acacia :
................................................................................................
Acamptoclados :
.
Acarreo :
................
Acento léxico :
.............................................
Achlaena :
.
Achmatherum :
.
Achneria :
.
Achyrodes :
.
Aciachne acicularis :
..............................
Acrobatidae :
...............
Acroceras :
....................................................................
Acróstico :
................
Activa tantum :

Actuación :
........................
Actual rey de Francia :
.
Ada :
................
Adeudo por domiciliación :
................
Adjetivo :
..........................................
Adstrato :
.

En la siguiente sesión veremos técnicas más sofisticadas de filtrado, pero por ahora se puede jugar con estas construcciones. Se puede seleccionar qué columnas se quiere mostrar e incluso filtros.

In [30]:
%%bash
cat <<EOF | ~/hbase/bin/hbase shell

scan 'wikilinks', {COLUMNS=>['to'], FILTER => "ColumnPrefixFilter('A')", LIMIT => 300}

EOF

HBase Shell; enter 'help<RETURN>' for list of supported commands.
Type "exit<RETURN>" to leave the HBase Shell
Version 1.1.6, r2039d967835afd066676b83f0bcb722cc55fdde3, Sun Aug 28 10:36:13 PDT 2016


scan 'wikilinks', {COLUMNS=>['to'], FILTER => "ColumnPrefixFilter('A')", LIMIT => 300}
ROW  COLUMN+CELL
 AGA column=to:Academia General del Aire, timestamp=1482443977312, value=
 AGA column=to:Asociaci\xC3\xB3n AGA, timestamp=1482443977312, value=
 ALGOL column=to:ABC ALGOL, timestamp=1482443977312, value=
 ALGOL column=to:Ada (lenguaje de programaci\xC3\xB3n), timestamp=1482443977312, value=Ada
 ALGOL column=to:Adriaan van Wijngaarden, timestamp=1482443977312, value=
 ALGOL column=to:Aritm\xC3\xA9tica de doble precisi\xC3\xB3n, timestamp=1482443977312, value=
 ASCII column=to:A, timestamp=1482443977335, value=
 ASCII column=to:ACK, timestamp=1482443977335, value=Acuse de recibo
 ASCII column=to:ACiD Productions, timestamp=1482443977335, value=
 ASCII column=to:ASCII extendido, timestamp=1

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/vagrant/hbase-1.1.6/lib/slf4j-log4j12-1.7.5.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/vagrant/hadoop-2.6.4/share/hadoop/common/lib/slf4j-log4j12-1.7.5.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


El proceso de `scan` recorre toda la tabla mostrando sólo las filas seleccionadas. HBase ofrece ciertas optimizaciones para que el escaneo sea eficiente, que veremos en la siguiente sesión.

Una introducción a los filtros y parámetros disponibles se puede ver [aquí](http://www.hadooptpoint.com/filters-in-hbase-shell/).

In [31]:
%%bash
cat <<EOF | ~/hbase/bin/hbase shell

scan 'wikipedia', {COLUMNS=>['revision'] , STARTROW => 'A', ENDROW=>'B'}

EOF

HBase Shell; enter 'help<RETURN>' for list of supported commands.
Type "exit<RETURN>" to leave the HBase Shell
Version 1.1.6, r2039d967835afd066676b83f0bcb722cc55fdde3, Sun Aug 28 10:36:13 PDT 2016


scan 'wikipedia', {COLUMNS=>['revision'] , STARTROW => 'A', ENDROW=>'B'}
ROW  COLUMN+CELL
 A fala column=revision:author, timestamp=1354140953, value=KLBot2
 A fala column=revision:comment, timestamp=1354140953, value=Bot: Cambiando #REDIRECT por #REDIRECCI\xC3\x93N.
 AGA column=revision:author, timestamp=1388068208, value=Savh
 AGA column=revision:comment, timestamp=1388068208, value=Savh movi\xC3\xB3 la p\xC3\xA1gina [[AGA (desambiguaci\xC3\xB3n)]] a [[AGA]]: Solicitado
 ALGOL column=revision:author, timestamp=1464261711, value=G\xC3\xBCnniX
 ALGOL column=revision:comment, timestamp=1464261711, value=/* Enlaces externos y bibliograf\xC3\xADa */
 ASCII column=revision:author, timestamp=1476477441, value=
 ASCII column=revision:comment, timestamp=1476477441, value=
 Abreviatura column=revi

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/vagrant/hbase-1.1.6/lib/slf4j-log4j12-1.7.5.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/vagrant/hadoop-2.6.4/share/hadoop/common/lib/slf4j-log4j12-1.7.5.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


## Prueba de búsquedas con filtros

A continuación se presentan un conjunto de ejercicios para ejercitar las búsquedas con filtros que se vieron en las sesiones de teoría. Se presentarán las búsquedas en código Python.

In [32]:
wikipedia = connection.table('wikipedia')
wikilinks = connection.table('wikilinks')
posts = connection.table('posts')
users = connection.table('users')

Ejemplo de la consulta anterior:

In [33]:
for key,data in wikipedia.scan(columns=['revision'], row_start='A', row_stop='B', limit=10):
    print key,'->',data

A fala -> {'revision:comment': 'Bot: Cambiando #REDIRECT por #REDIRECCI\xc3\x93N.', 'revision:author': 'KLBot2'}
AGA -> {'revision:comment': 'Savh movi\xc3\xb3 la p\xc3\xa1gina [[AGA (desambiguaci\xc3\xb3n)]] a [[AGA]]: Solicitado', 'revision:author': 'Savh'}
ALGOL -> {'revision:comment': '/* Enlaces externos y bibliograf\xc3\xada */', 'revision:author': 'G\xc3\xbcnniX'}
ASCII -> {'revision:comment': '', 'revision:author': ''}
Abreviatura -> {'revision:comment': '/* Diccionarios de abreviaturas */', 'revision:author': ''}
Acacia -> {'revision:comment': 'Revertidos los cambios de [[Special:Contributions/212.55.25.241|212.55.25.241]] ([[User talk:212.55.25.241|disc.]]) a la \xc3\xbaltima edici\xc3\xb3n de Jkbw', 'revision:author': 'Jkbw'}
Acamptoclados -> {'revision:comment': 'Bot: Cambiando #REDIRECT por #REDIRECCI\xc3\x93N.', 'revision:author': 'KLBot2'}
Acarreo -> {'revision:comment': 'Revertidos los cambios de [[Special:Contributions/~Expresses life|~Expresses life]] ([[User talk:~Ex

Y con filtro (no produce ningún resultado). ¿Por qué? Porque la cláusula row_stop es exclusiva; es decir, excluye a todas las filas que empiecen a partir de 'B'.

In [172]:
for key,data in wikipedia.scan(columns=['revision'], row_start='A', row_stop='B', \
                               filter="PrefixFilter('B')", limit=10):
    print key,'->',data

## EJERCICIO: Mostrar la familia de columnas `revision` para la entrada `ASCII` de la tabla `wikipedia`.

In [39]:
for key,data in wikipedia.scan(columns=['revision'], row_start='ASCII', row_stop='ASCII', limit=10):
    print key,'->',data

ASCII -> {'revision:comment': '', 'revision:author': ''}


## EJERCICIO: Mostrar las 20 primeras filas de la tabla `wikipedia` cuyas columnas empiecen por `com`.

In [171]:
for key,data in wikipedia.scan(columns=None, row_start='A', row_stop='B', \
                               filter="ColumnPrefixFilter('com')", limit=20):
    print key,'->',data

A fala -> {'revision:comment': 'Bot: Cambiando #REDIRECT por #REDIRECCI\xc3\x93N.'}
AGA -> {'revision:comment': 'Savh movi\xc3\xb3 la p\xc3\xa1gina [[AGA (desambiguaci\xc3\xb3n)]] a [[AGA]]: Solicitado'}
ALGOL -> {'revision:comment': '/* Enlaces externos y bibliograf\xc3\xada */'}
ASCII -> {'revision:comment': ''}
Abreviatura -> {'revision:comment': '/* Diccionarios de abreviaturas */'}
Acacia -> {'revision:comment': 'Revertidos los cambios de [[Special:Contributions/212.55.25.241|212.55.25.241]] ([[User talk:212.55.25.241|disc.]]) a la \xc3\xbaltima edici\xc3\xb3n de Jkbw'}
Acamptoclados -> {'revision:comment': 'Bot: Cambiando #REDIRECT por #REDIRECCI\xc3\x93N.'}
Acarreo -> {'revision:comment': 'Revertidos los cambios de [[Special:Contributions/~Expresses life|~Expresses life]] ([[User talk:~Expresses life|disc.]]) a la \xc3\xbaltima edici\xc3\xb3n de Farisori'}
Acento léxico -> {'revision:comment': 'Peque\xc3\xb1as correcciones [[WP:CEM]].'}
Achlaena -> {'revision:comment': 'Bot: Cam

## EJERCICIO: Mostrar las 20 primeras filas de la tabla `wikipedia` cuyas columnas empiecen por `com` y la clave de columna empieza por '`B`'.

In [170]:
for key,data in wikipedia.scan(columns=None, row_start=None, row_stop=None, row_prefix='B', \
                               filter="ColumnPrefixFilter('com')", limit=20):
    print key,'->',data

BASIC -> {'revision:comment': 'Correcciones ortogr\xc3\xa1ficas'}
BIOS -> {'revision:comment': 'Revertidos los cambios de [[Special:Contributions/79.153.162.34|79.153.162.34]] ([[User talk:79.153.162.34|disc.]]) a la \xc3\xbaltima edici\xc3\xb3n de Mansoncc'}
Bacteria -> {'revision:comment': 'Revertidos los cambios de [[Special:Contributions/181.57.143.98|181.57.143.98]] ([[User talk:181.57.143.98|disc.]]) a la \xc3\xbaltima edici\xc3\xb3n de Osado'}
Balanophoraceae -> {'revision:comment': ''}
Baloncesto -> {'revision:comment': 'Revertidos los cambios de [[Special:Contributions/64.32.117.118|64.32.117.118]] ([[User talk:64.32.117.118|disc.]]) a la \xc3\xbaltima edici\xc3\xb3n de Wiki LIC'}
Balsaminaceae -> {'revision:comment': 'Bot:Reparando enlaces'}
Bambuseae -> {'revision:comment': ''}
Bandera -> {'revision:comment': 'Revertidos los cambios de [[Special:Contributions/Mariiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii|Mariiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii

## EJERCICIO: Mostrar sólo la columna `revision:author` de las filas de la tabla `wikipedia` cuya clave empiece por `a` y termine por `a` (obviando mayúsculas y minúsculas).

In [169]:
for key,data in wikipedia.scan(columns=['revision:author'], row_start=None, row_stop=None, \
                               filter="RowFilter(=, 'regexstring:^(a|A).*(a|A)$')"):
    print key,'->',data

A fala -> {'revision:author': 'KLBot2'}
AGA -> {'revision:author': 'Savh'}
Abreviatura -> {'revision:author': ''}
Acacia -> {'revision:author': 'Jkbw'}
Achlaena -> {'revision:author': 'KLBot2'}
Achneria -> {'revision:author': 'MILEPRI'}
Actual rey de Francia -> {'revision:author': 'CEM-bot'}
Ada -> {'revision:author': 'Solbaken'}
Adventista -> {'revision:author': 'Benjavalero'}
Afrodita -> {'revision:author': 'Foundling'}
Agricultura -> {'revision:author': 'Foundling'}
Agricultura ecológica -> {'revision:author': 'Lin linao'}
Aira -> {'revision:author': 'BenjaBot'}
Airopsis tenella -> {'revision:author': 'BenjaBot'}
Ala -> {'revision:author': 'Shooke'}
Albania -> {'revision:author': ''}
Alberta -> {'revision:author': 'Alexandra Desir\xc3\xa9e Reyes'}
Aldea -> {'revision:author': '4lextintor'}
Alegoría -> {'revision:author': 'Arjuno3'}
Alemania -> {'revision:author': 'Arjuno3'}
Alergia -> {'revision:author': 'Jkbw'}
Alfaroa -> {'revision:author': 'Macofe'}
Algoritmo de compresión con pé

## EJERCICIO: Mostrar las filas de la tabla `wikipedia` cuya clave contenga al menos un número.

In [184]:
for key,data in wikipedia.scan(columns=['revision'], row_start=None, row_stop=None, \
                               filter="RowFilter(=, 'regexstring:[0-9]+$')"):
    print key,'->',data

Año 1900 -> {'revision:comment': 'Deshecha la edici\xc3\xb3n 6633628 de [[Special:Contributions/Bark|Bark]] ([[Usuario Discusi\xc3\xb3n:Bark|disc.]])', 'revision:author': 'Bark~eswiki'}
Commodore 128 -> {'revision:comment': 'Correcciones ortogr\xc3\xa1ficas', 'revision:author': 'Benjavalero'}
Commodore 64 -> {'revision:comment': 'Correcci\xc3\xb3n. Cambio del 69 al 64.', 'revision:author': ''}
Commodore Amiga 500 -> {'revision:comment': 'Deshecha la edici\xc3\xb3n 80941305 de [[Especial:Contribuciones/86.191.111.137|86.191.111.137]] ([[Usuario Discusi\xc3\xb3n:86.191.111.137|disc.]])', 'revision:author': ''}
Constitución de 1812 -> {'revision:comment': 'Repetida', 'revision:author': 'ManuelGR'}


## EJERCICIO: Mostrar las filas de la tabla `wikipedia` cuyo autor de revisión sea `Addbot`.

In [187]:
for key,data in wikipedia.scan(columns=['revision'], row_start=None, row_stop=None, \
                               filter="SingleColumnValueFilter('revision', 'author', =, 'binary:Addbot')"):
    print key,'->',data

Discolichenes -> {'revision:comment': 'Moviendo 1 enlace(s) interling\xc3\xbc\xc3\xadstico(s), ahora proporcionado(s) por [[d:|Wikidata]] en la p\xc3\xa1gina [[d:q8560354]].', 'revision:author': 'Addbot'}
Euchlaena -> {'revision:comment': 'Moviendo 1 enlace(s) interling\xc3\xbc\xc3\xadstico(s), ahora proporcionado(s) por [[d:|Wikidata]] en la p\xc3\xa1gina [[d:q8843748]].', 'revision:author': 'Addbot'}
Wikipedia:Clasificación Unesco de 6 dígitos/21 Astronomía y Astrofísica -> {'revision:comment': '[[User:addbot|Bot]]: Moviendo enlace(s) interling\xc3\xbc\xc3\xadstico(s), ahora proporcionado(s) por [[d:|Wikidata]] en la p\xc3\xa1gina [[d:q11932575]]', 'revision:author': 'Addbot'}
Wikipedia:Clasificación Unesco de 6 dígitos/24 Ciencias de la Vida -> {'revision:comment': '[[User:addbot|Bot]]: Moviendo enlace(s) interling\xc3\xbc\xc3\xadstico(s), ahora proporcionado(s) por [[d:|Wikidata]] en la p\xc3\xa1gina [[d:q11932577]]', 'revision:author': 'Addbot'}
Wikipedia:Clasificación Unesco de 6

## EJERCICIO: Mostrar las filas de la tabla `wikipedia` tales que alguno de sus valores de campos de columnas sea menor que `1`.

In [201]:
for key,data in wikipedia.scan(columns=['revision'], row_start=None, row_stop=None, \
                               filter="ValueFilter(<, 'binary:1')"):
    print key,'->',data

ALGOL -> {'revision:comment': '/* Enlaces externos y bibliograf\xc3\xada */'}
ASCII -> {'revision:comment': '', 'revision:author': ''}
Abreviatura -> {'revision:comment': '/* Diccionarios de abreviaturas */', 'revision:author': ''}
Acroceras -> {'revision:comment': '/* Taxonom\xc3\xada */ ; ortograf\xc3\xada'}
Acróstico -> {'revision:comment': '', 'revision:author': ''}
Adeudo por domiciliación -> {'revision:comment': '(Bot) Normalizaci\xc3\xb3n de fechas'}
Adstrato -> {'revision:comment': '/* Los resultos */'}
Advanced Interactive Executive -> {'revision:comment': '/* Versiones */'}
Aegilops -> {'revision:comment': '(Bot) Normalizaci\xc3\xb3n de fechas; cambios triviales'}
Agujero negro de Schwarzschild -> {'revision:comment': '', 'revision:author': ''}
Aira -> {'revision:comment': '(Bot) Normalizaci\xc3\xb3n de fechas; cambios triviales'}
Airopsis tenella -> {'revision:comment': '(Bot) Normalizaci\xc3\xb3n de fechas; cambios triviales'}
Ala -> {'revision:comment': '/* Deporte */'}
Al

## EJERCICIO: Mostrar las filas de la tabla `users` (sólo la columna `rawdata:Location`) de usuarios de España (se supondrá que su localización (columna `rawdata:Location`) contiene `España` o `ES`, obviando mayúsculas y minúsculas).

In [208]:
for key,data in users.scan(columns=['rawdata:Location'], row_start=None, row_stop=None, \
                               filter="SingleColumnValueFilter('rawdata', 'Location', =, 'regexstring:(España|españa|Spain|spain)+|(\bES\b|\bes\b)+')"):
    print key,'->',data

10000 -> {'rawdata:Location': 'Barcelona, Spain'}
10005 -> {'rawdata:Location': 'Spain'}
10015 -> {'rawdata:Location': 'San Cristobal De La Laguna, Spain'}
10048 -> {'rawdata:Location': 'Spain'}
10049 -> {'rawdata:Location': 'Spain'}
10085 -> {'rawdata:Location': 'Alicante, Spain'}
10089 -> {'rawdata:Location': 'Spain'}
1011 -> {'rawdata:Location': 'Madrid, Spain'}
10133 -> {'rawdata:Location': 'Seville, Spain'}
10136 -> {'rawdata:Location': 'Madrid, Espa\xc3\xb1a'}
10143 -> {'rawdata:Location': 'Majorca, Balearic Islands, Spain'}
10144 -> {'rawdata:Location': 'Spain'}
10158 -> {'rawdata:Location': 'Spain'}
10175 -> {'rawdata:Location': 'Girona, Spain'}
102 -> {'rawdata:Location': 'Granada, Spain'}
10333 -> {'rawdata:Location': 'Spain'}
10335 -> {'rawdata:Location': 'Matar\xc3\xb3, Espa\xc3\xb1a'}
10338 -> {'rawdata:Location': 'Madrid, Spain'}
10392 -> {'rawdata:Location': 'Spain'}
10438 -> {'rawdata:Location': 'Seville, Spain'}
10440 -> {'rawdata:Location': 'Spain'}
10441 -> {'rawdata

## EJERCICIO: Comparar si hay más usuarios de Santiago de Compostela que de Murcia :).

In [222]:
santiago = users.scan(columns=['rawdata:Location'], row_start=None, row_stop=None, \
                               filter="SingleColumnValueFilter('rawdata', 'Location', =, 'regexstring:(Santiago|santiago|Compostela|compostela)+$')")
num_santiago = 0
for usuario in santiago:
    num_santiago += 1
print(num_santiago)
murcia = users.scan(columns=['rawdata:Location'], row_start=None, row_stop=None, \
                               filter="SingleColumnValueFilter('rawdata', 'Location', =, 'regexstring:(Murcia|murcia)+$')")
num_murcia = 0
for usuario in murcia:
    num_murcia += 1
print(num_murcia)

print ("Hay más usuarios de Santiago " + str(num_santiago) + " que de Murcia " + str(num_murcia))

5
2
Hay más usuarios de Santiago 5 que de Murcia 2


## EJERCICIO: Mostrar las filas de la tabla `posts` que hacen referencia al _tag_ "clojure".

In [231]:
for key,data in posts.scan(columns=['rawdata:Tags'], row_start=None, row_stop=None, \
                               filter="SingleColumnValueFilter('rawdata', 'Tags', =, 'regexstring:clojure+')"):
    print key,'->',data

1422 -> {'rawdata:Tags': '<clojure>'}


## EJERCICIO (opcional): Crear una nueva tabla `poststags` que, de forma eficiente, para cada _tag_, liste los `Id` de los _posts_ que utilizan ese _tag_.

**Nota: El código de ejemplo para separar todas las etiquetas del campo `Tags` de un `post` está en la [sesión 1](../sql/sesion1.ipynb#Código-de-suma-de-posts-de-cada-Tag)**.